## 12 생성모델을 위한 딥러닝
12.1 텍스트 생성 <br>
12.2 딥드림 <br>
12.3 뉴럴 스타일 트랜스퍼 <br>
12.4 변이형 오토인코더를 사용한 이미지 생성 <br>
12.5 생성적 적대 신경망 소개 <br>
12.6 요약 <br><br>

### 12.1 텍스트 생성
텍스트 생성을 예로 드지만 동일한 기법으로 어떤 종류의 시퀀스 데이터도 생성할 수 있음. <br><br>

#### 12.1.1 시퀀스 생성을 위한 딥러닝 모델의 간단한 역사 
2014년까지만 해도 LSTM은 주류가 아니었음 <br>
2002년 더글라스 에크는 스위스의 슈미드후버의 연구실에서 LSTM을 음악 생성에 처음 적용하여 가능성 있는 결과를 얻었음 <br><br>

#### 12.1.2 시퀀스 데이터를 어떻게 생성할까? 
이전 토큰을 입력으로 사용해서 시퀀스의 다음 1개 또는 몇개의 토큰을 (transformer나 RNN으로) 예측하는 것이다. <br>
다음 토큰의 확률을 모델링할 수 있는 네트워크를 <b>언어모델(language model)</b> 이라고 부른다. <br>

언어 모델은 언어의 통계적 구조인 잠재 공간을 탐색한다. 언어 모델을 훈련하고 나면 이 모델에서 샘플링을 할 수 있다. (=새로운 시퀀스를 생성한다.) 초기 텍스트 문자열을 주입하고 (조건 데이터(conditioning data)라고 부른다. 새로운 글자나 단어를 생성한다(한번에 여러개의 토큰을 생성할 수도 있다.) 생성된 출력은 다시 입력 데이터로 추가됨 <br><br>

#### 12.1.3 샘플링 전략의 중요성
텍스트를 생성할 떄 다음 문자를 선택하는 방법은 아주 중요하다.
탐욕적 샘플링(greedy sampling), 반복적이고 예상 가능한 문자열을 만들기 떄무에 논리적인 어처럼 보이지 않는다. <br>
확률적 샘플링(stochastic sampling)은 머신러닝에서 확률적(stochastic)이란 뜻은 무작위(random)하다는 의미이다. <br><br>

#### 코드 12-1 다른 온도 값을 사용하여 확률 분포의 가중치 바꾸기

In [1]:
import numpy as np

def reweight_distribution(original_distribution, temperature=0.5):  
    # original_distribution은 전체 합이 1인 1D 넘파이 배열임. temperature는 출력 분포의 엔트로피의 양을 결정한다.
    distribution = np.log(original_distribution)/temperature
    distribution = np.exp(distribution)
    
    return distribution/np.sum(distribution)
    # 원본 분포의 가중치를 변경하며 반환한다. 이 분포의 합은 1이 아닐 수 있으므로 새로운 분포의 합으로 나눈다. 

#### 12.1.4 케라스를 사용한 텍스트 생서 모델 구현

이번 예시는 이전에 본 적 없는 영화 리뷰를 생성하는 방법을 학습시켜 보겠다. 따라서 이 언어 모델은 일반적ㅇ니 영어를 모델링하는 것이 아니라 이런 영화 리뷰의 스타일과 주제를 모델링할 것임.

#### 코드 12-2 IMDB 영화 리뷰 데이터셋 내려받아 압축 풀기

In [2]:
!wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

--2023-08-09 09:50:54--  https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz.1’

aclImdb_v1.tar.gz.1 100%[===================>]  80.23M  51.4MB/s    in 1.6s    

2023-08-09 09:50:56 (51.4 MB/s) - ‘aclImdb_v1.tar.gz.1’ saved [84125825/84125825]



aclImdb안에는 2개의 서브 폴더가 있음. 한 폴더는 부정적인 영화 리뷰를 담고 있고 다른 폴더는 긍정적인 영화 리뷰를 담고 있다. 리뷰마다 하나의 텍스트 파일로 구성됨.

#### 코드 12-3 텍스트 파일(한 파일 = 한 샘플)에서 데이터셋 만들기

In [3]:
import tensorflow as tf
from tensorflow import keras

dataset = keras.utils.text_dataset_from_directory(
    directory = "aclImdb", label_mode=None, batch_size=256)
dataset = dataset.map(lambda x:tf.strings.regex_replace(x, "<br />", " "))
# 이 리뷰에 많이 등장하는 <br /> HTML태그를 제거한다. 
# 텍스트 분류 작업에서는 중요하지 않지만 이 예제에서는 <br /> 태그를 생성하고 싶지 않기 떄문이다!

Found 100006 files belonging to 1 classes.


이제 TextVectorization층을 사용하여 이 예제에서 사용할 어휘 사전을 만든다. 각 리뷰에서 처음 sequence_length개 단어만 사용하겠다. 즉, TextVectorization층은 텍스트를 벡터화할 때 이보다 긴 리뷰 내용을 잘라 버린다. 

#### 코드 12-4 TextVectorization층 준비하기

In [4]:
from tensorflow.keras.layers import TextVectorization

sequence_length = 100
vocab_size = 15000  # 가장 자주 등장하는 1만5천개의 단어만 사용하겠다. 그 외 단어는 모두 OOV 토큰인 "[UNK]"로 처리
text_vectorization = TextVectorization(  
    max_tokens = vocab_size,
    output_mode = "int",  # 정수 단어 인덱스의 시퀀스를 반환하도록 설정한다.
    output_sequence_length = sequence_length  # 길이가 100인 입력과 타깃을 사용한다 
    # (타깃은 한 스텝 차이가 나기 때문에 실제로 모델은 99개의 단어 시퀀스를 보게 된다)
)
text_vectorization.adapt(dataset)

이 층을 사용하여 언어 모델링 데이터셋을 만들어 보겠음, 입력 샘플은 벡터화된 텍스트고 타깃은 한 스텝앞의 동일 텍스트임.

#### 코드 12-5 언어 모델링 데이터셋 만들기

In [5]:
def prepare_lm_dataset(text_batch):
    vectorized_sequences = text_vectorization(text_batch)
    # 텍스트(문자열)의 배치를 정수 시퀀스의 배치로 변환한다.
    x = vectorized_sequences[:, :-1]  # 시퀀스의 마지막 단어를 제외한 입력을 만든다.
    y = vectorized_sequences[:, 1:]  # 시퀀스의 첫 단어를 제외한 타깃을 만든다.
    return x, y

lm_dataset = dataset.map(prepare_lm_dataset, num_parallel_calls=4)

#### 트랜스포머 기반의 시퀀스-투-시퀀스 모델

10장의 온도 예측 문제에서 했던 것처럼 N개 단어의 시퀀스를 입력으로 받아 N+1번째 단어를 예측하는 모델을 훈련한다. 하지만 시퀀스 생성 고나점으로 보았을 때 여기에는 몇가지 이슈가 있다. <br>

첫째, 이 모델은 N개의 단어로 예측을 만드는 방법을 학습하지만 N개보다 적은 단어로 예측을 시작할 수 있어야 한다. <br>
둘째, 훈련에 사용하는 많은 시퀀스는 중복되어 있다. <br>

이런 두 이슈를 해결하기 위해 seq-to-seq 모델을 사용하겠다. 즉, 단어 N개의 시퀀스(0 에서 N까지)를 모델에 주입하고 한 스텝 다음의 시퀀스 (1에서 N+1까지)를 예측할거임. <br>

텍스트 생성에는 소스 시퀀스가 없다. 과거 토큰이 주어지면 타깃 시퀀스에 있는 다음 토큰을 에측하는 것 뿐. 따라서 이 작업은 디코더만 사용해서 수해할 수 있음. Causal padding덕분에 디코더는 단어 N+1을 예측하기 위해 0...N만 바라볼 것임 <br>

In [6]:
import tensorflow as tf
from tensorflow.keras import layers

class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
          num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
          num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super(TransformerDecoder, self).get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

#### 코드 12-6 간단한 트랜스포머 기반 언어 모델

In [7]:
from tensorflow.keras import layers
embed_dim = 256
latent_dim = 2048
num_heads = 2

inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, x)
outputs = layers.Dense(vocab_size, activation="softmax")(x)     
# 출력 시퀀스 타임스텝마다 가능한 어휘 사전의 단어에 대해 소프트맥스 확률을 계산한다
model = keras.Model(inputs, outputs)
model.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop")

#### 12.1.5 가변 온도 샘플링을 사용한 텍스트 생성 콜백

콜백을 사용하여 에포크가 끝날 때마다 다양한 온도로 텍스트를 생성하겠다. 모델이 수렴하면서 생성된 텍스트가 어떻게 발전하는지와 온도가 샘플링 전략에 미치는 영향을 확인할 수 있다. 즉, 이 두 단어를 시작으로 모든 텍스트를 생성하겠다. <br>

#### 코드 12-7 텍스트 생성 콜백

In [8]:
import numpy as np

tokens_index = dict(enumerate(text_vectorization.get_vocabulary()))
# 단어 인덱스를 문자열로 매핑하는 딕셔너리이다. 텍스트 디코딩에 사용한다.

def sample_next(predictions, temperature=1.0): # 어떤 확률 분포에 대한 가변 온도 샘플링을 구현한다.
    predictions = np.asarray(predictions).astype("float64")
    predictions = np.log(predictions) / temperature
    exp_preds = np.exp(predictions)
    predictions = exp_preds / np.sum(exp_preds)
    probabs = np.random.multinomial(1, predictions, 1)
    return np.argmax(probas)

class TextGenerator(keras.callbacks.Callback):
    def __init__(self, 
                prompt, 
                generate_length,
                model_input_length,
                temperatures=(1.,),
                print_freq =1):
        self.prompt = prompt
        self.generate_length = generate_length
        self.model_input_length = model_input_length
        self.temperatures = temperatures
        self.pring_freq = print_freq

def on_epoch_end(self, epoch, logs=None):
    if (epoch+1)% self.print_freq != 0:
        return
    for temprature in self.temperatures:
        print("== Generating with temperature", temperature)
        sentence = self.prompt  # 시작 단어에서부터 텍스트를 생성
        for i in range(self.generate_length):
            tokenized_sentence = text_vectorization([sentence])
            predictions = self.model(tokenized_sentence)
            next_token = sample_next(predictions[0, i, :])
            sampled_token = tokens_index[next_token]
            sentence += " " + sampled_token
        print(setnence)
    
prompt = "This movie"
text_gen_callback = TextGenerator(prompt, generate_length = 50, 
                                   model_input_length = sequence_length, temperatures=(0.2, 0.5, 0.7, 1., 1.5) )

fit() 메서드를 호출하겠음

#### 코드 12-8 언어 모델 훈련하기

In [ ]:
model.fit(lm_dataset, epochs=200, callbacks=[text_gen_callback])

Epoch 1/200
391/391 [==============================] - 177s 391ms/step - loss: 5.3539
Epoch 2/200
391/391 [==============================] - 158s 405ms/step - loss: 4.8133
Epoch 3/200
391/391 [==============================] - 159s 406ms/step - loss: 4.5913
Epoch 4/200
391/391 [==============================] - 159s 405ms/step - loss: 4.4649
Epoch 5/200
391/391 [==============================] - 159s 406ms/step - loss: 4.3781
Epoch 6/200
391/391 [==============================] - 159s 406ms/step - loss: 4.3119
Epoch 7/200
391/391 [==============================] - 159s 406ms/step - loss: 4.2588
Epoch 8/200
391/391 [==============================] - 159s 405ms/step - loss: 4.2149
Epoch 9/200
391/391 [==============================] - 159s 406ms/step - loss: 4.1776
Epoch 10/200
391/391 [==============================] - 159s 406ms/step - loss: 4.1451
Epoch 11/200
391/391 [==============================] - 159s 406ms/step - loss: 4.1168
Epoch 12/200
391/391 [==============================

391/391 [==============================] - 159s 406ms/step - loss: 3.7030
Epoch 96/200
391/391 [==============================] - 159s 406ms/step - loss: 3.7013
Epoch 97/200
391/391 [==============================] - 159s 406ms/step - loss: 3.6999
Epoch 98/200
391/391 [==============================] - 159s 406ms/step - loss: 3.6982
Epoch 99/200
391/391 [==============================] - 159s 406ms/step - loss: 3.6968
Epoch 100/200
391/391 [==============================] - 159s 406ms/step - loss: 3.6949
Epoch 101/200
391/391 [==============================] - 160s 408ms/step - loss: 3.6936
Epoch 102/200
391/391 [==============================] - 159s 406ms/step - loss: 3.6919
Epoch 103/200
391/391 [==============================] - 160s 408ms/step - loss: 3.6903
Epoch 104/200
391/391 [==============================] - 159s 406ms/step - loss: 3.6885
Epoch 105/200
391/391 [==============================] - 159s 406ms/step - loss: 3.6872
Epoch 106/200
391/391 [===========================

### 12.2 딥드림

딥드림(DeepDream)은 합성곱 신경망이 학습한 표현을 사용하여 예쑬적으로 이미지를 조작하는 기법이다. 
컨브넷 상위 층에 있는 특정 필터의 활성화를 극대화하기 위해 컨브넷의 입력에 경사 상법을 적용했음. 몇 개의 사소한 차이를 빼면 딥드림도 동일한 아이디어를 사용한다. 
- 딥드림에서는 특정 필터가 아니라 전체 층의 활성화를 최대화한다. 한꺼번에 많은 특성을 섞어 시각화한다. <br>
- 빈 이미지나 노이즈가 조금 있는 입력이 아니라 이미 가지고 있는 이미지를 사용한다. 그 결과 기존 시각 패턴을 바탕으로 이미지의 요소들을 다소 예술적인 스타일로 왜곡시킨다. <br>
- 입력 이미지는 시각 품질을 높이기 위해 여러 다른 스케일(옥타브(Octave)라고 부른다)로 처리한다. <br>

#### 12.2.1 케라스 딥드림 구현
딥드림 이미지를 만들어 보자.

#### 코드 12-9 테스트 이미지 내려받기

In [ ]:
from tensorflow import keras
import matplotlib.pyplot as plt

base_image_path = keras.utils.get_file(
    "coast.jpg", origin="https://img-datasets.s3.amazonaws.com/coast.jpg")

plt.axis("off")
plt.imshow(keras.utils.load_img(base_image_path))

실제로 인셉션이 딥드림 이미지를 잘 만듦

#### 코드 12-10 사전 훈련된 InceptionV3 모델 로드하기

In [ ]:
from tensorflow.keras.applications import inception_v3

model = inception_v3.InceptionV3(weights="imagenet", include_top = False)

사전 훈련된 컨브넷을 사용하여 코드 12-11과 같이 다양한 중간층의 활성화를 반환하는 특성 추출 ㅗㅁ델을 만들겠음.

#### 코드 12-11 딥드림 손실에 대한 각 층의 기여도 설정하기

In [ ]:
layer_settings = {  # 활성화를 최대화할 층과 전체 손시에 대한 가중치. 이 설정을 바꾸면 새로운 시각 효과를 얻을 수 있음.
    "mixed4": 1.0,
    "mixed5": 1.5,
    "mixed6": 2.0,
    "mixed7": 2.5,
}

outputs_dict = dict(  # 각 층의 심볼릭 출력
    [  (layer.name, layer.output)
        for layer in [model.get_layer(name)
                     for name in layer_settings.keys()] ])

feature_extractor = keras.Model(inputs=model.inputs, outputs=outputs_dict)
# 각 타깃 층의 활성화 값을 (하나의 딕셔너리로) 반환하는 모델

#### 코드 12-12 딥드림 손실

In [ ]:
def compute_loss(input_image):
    features = feature_extractor(input_image)  # 활성화를 추출
    loss = tf.zeros(shape=())  # 손실을 0으로 초기화
    for name in features.keys():
        coeff = layer_settings[name]
        activation = features[name]
        loss += coeff*tf.reduce_mean(tf.square(activation[:, 2:-2, 2:-2, :]))  
        # 경계 부근의 인공적인 패턴을 피하기 위해 테두리가 아닌 픽셀만 손실에 추가한다.
    return loss

딥드림 알고리즘은 필터 시각화의 다중 스케일 버전임

#### 코드 12-13 딥드림 경사 상승법 단계

In [ ]:
import tensorflow as tf

@ tf.function
def gradient_ascent_step(image, learning_rate):
    with tf.GradientTape() as tape:  # 현재 임지에 대한 딥드림 손실의 그레디언트를 계산
        tape.watch(image)
        loss = compute_loss(image)
    grads = tape.gradient(loss, image)  
    grads = tf.math.l2_normalize(grads)  # gradient를 정규화한다 
    image += learning_rate*grads
    return loss, image

def gradient_ascent_loop(image, iterations, learning_rate, max_loss=None):
    # 주어진 이미지 스케일(옥타브)에 대한 경사 상승법을 수행한다.
    for i in range(iterations):
        loss, image = gradient_ascent_step(image, learning_rate)
        if max_loss is not None and loss > max_loss: 
            # 손실이 일정 임계 값을 넘으면 중지한다 (과도하게 최적화하면 원치 않는 이미지를 만들 수 있다.)
            break
        print(f"... 스텝 {i}에서 손실 값: {loss:.2f}") 
    return image

In [ ]:
step = 20.  # 경사 상승법 단계 크기
num_octave = 3  # 경사 상승법을 실행할 스케일 횟수
octave_scale = 1.4  # 연속적인 스케일 사이의 크기 비율
iterations = 30  # 스케일 단계마다 수행할 경사 상승법 단계 횟수
max_loss = 15. # 이보다 손실이 커지면 현재 스케일에서 경사 상승법 과정을 중지

#### 코드 12-14 이미지 처리 유틸리티

In [ ]:
import numpy as np

def preprocess_image(image_path): # 이미지를 로드하고, 크기를 바꾸어 적절한 배열로 변환하는 유틸리티 함수
    img = keras.utils.load_img(image_path)
    img = keras.utils.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = keras.applications.inception_v3.preprocess_input(img)
    return img

def preprocess_image(img):  # 넘파이 배열을 이미지로 변환하는 유틸리티 함수
    img = img.reshape((img.shape[1], img.shape[2], 3))
    img += 1.0   # InceptionV3 전처리 복원하기
    img *= 127.5
    img = np.clip(img, 0, 255).astype("uint8")   # uint8로 바꾸고 [0, 255]범위로 클리핑한다.
    return img 

이것이 바깥쪽 루프, 스케일을 연속적으로 증가시키면서 (점점 뭉개지거나 픽셀 경계가 나타나므로) 이미지 디테일을 많이 잃지 않도록 간단한 기교를 사용할 수 있음. 

#### 코드 12-15 연속적인 여러개의 '옥타브'에 걸쳐 경사 상승법 실행하기

In [ ]:
original_img = preprocess_image(base_image_path)  # 테스트 이미지를 로드한다.
original_shape = original_img.shape[1:3]


successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim/(octave_scale**i)) for dim in original_shape])
    successive_shapes.append(shape)
successive_shapes = successive_shapes[::-1]
shrunk_original_img = tf.image.resize(original_img, successive_shapes[0])

img = tf.identity(original_img)  # 이미지를 복사한다(원본 이미지는 그대로 보관한다)
for i, shape in enumerate(successive_shapes): # 여러 옥타브에 대해 반복
    print(f"{shape} 크기의 {i}번째 옥타브 처리")
    img = tf.image.resize(img, shape)  # 딥드림 이미지의 스케일을 높인다.
    img = gradient_ascent_loop(   # 경사 상승법을 실행하고 딥드림 이미지를 수정한다.
            img, iterations = iterations, 
            learnint_rate = step, max_loss = max_loss
    )
    upscaled_shrunk_original_img = tf.image.resize(shrunk_original_img, shape) 
    # 작은 버전의 원본 이미지의 스케일을 높인다. 픽셀 경계가 보일것이다.
    
    same_size_original = tf.image.resize(original_img, shape)  # 이 크기에 해당하는 고해상도 버전의 원본 이미지를 계산한다.
    lost_detail = same_size_original - upscaled_shrunk_original_img  # 두 이미지의 차이가 스케일을 높였을 때 손실된 디테일이다.
    img += lost_detail # 손실된 디테일을 딥드림 이미지에 다시 주입한다.
    shrunk_original_img = tf.image.resize(original_img, shape)

keras.utils.save_img("dream.png", deprocess_image(img.numpy()))  # 최종 결과를 저장한다.

__cf) Note__

원본 인셉션 V3 네트워크는 299 * 299크기의 이미지에서 훈련됐음. 이런 이유 때문에 이 딥드림 구현은 이미지 크기를 줄이는 정도가 적당한 300 * 300과 400 * 400 사이에 있는 이미지에서 훨씬 좋은 결과를 만든다. 어떤 크기나 비율을 가진 이미지에서도 이 코드를 실행할 수 있다. <br><br>

#### 12.2.2 정리
- 딥드림은 네트워크가 학습한 표현을 기반으로 컨브넷을 거꾸로 실행하여 입력 이미지를 생성한다. <br>
- 재미있는 결과가 만들어지고, 때로는 환각제 때문에 시야가 몽롱해진 사람이 만든 이미지 같음 <br>
- 이 과정은 이미지 모델이나 컨브넷에 국한되지 않는다. 음성, 음악등에도 적용될 수 있음 <br><br>


### 12.3 뉴럴 스타일 트랜스퍼
딥드림 이외에 딥러닝을 사용하여 이미지를 변경하는 또 다른 주요 분야는 뉴럴 스타일 트랜스퍼(Neural Style Transfer). <br> 
목표를 표현한 손실 함수를 정의하고 이 손실을 최소화한다. 여기에서 원하는 것은 다음과 같다. 참조 이미지의 스타일을 적용하면서 원본 이미지의 콘텐츠를 보존하는 것이다. 콘텐츠와 스타일을 수학적으로 정의할 수 있다면 최소화할 손실함수는 다음과 같을 것이다.

In [ ]:
loss = distance(style(reference_image)-style(combination_image)) +
        distance(content(original_image)- content(combination_image))

여기에서 distance는 L2노름 같은 노름 함수. content 함수는 이미지의 콘텐츠 표현을 계산한다. <br><br>


#### 12.3.1 콘텐츠 손실
하위 층의 활성화: 네트워크에 있는 하위층의 활성화는 이미지에 관한 국부적인 정보를 담고 있다. <br>
상위 층의 활성화: 점점 전역적이고 추상적인 정보를 담게 된다. <br>
다른 방식으로 생각하면 컨브넷 층의 활성화는 이미지를 다른 크기의 콘텐츠로 분해한다고 볼 수 있다. <br><br>

#### 12.3.2 스타일 손실
콘텐츠 손실은 하나의 상위 층만 사용, <br>
게티스 등이 정의한 스타일 손실은 컨브넷의 여러 층을 사용한다. 게티스 등은 층의 활성화 출력의 그람행렬(Gram Matrix)을 스타일 손실로 사용했다. 그람 행렬은 층의 특성 맵들의 내적(inner dot)이다. 

사전 훈련된 컨브넷을 사용하여 다음 손실들을 정의할 수 있다. <br>
- 콘텐츠를 보존하기 위해 원본 이미지와 생성된 이미지 사이에서 상위 층의 활성화를 비슷하게 유지한다. 이 컨브넷은 원본 이미지와 생성된 이미지에서 동일한 것을 봐야 한다. <br>
- 스타일을 보존하기 위해 저수준 층과 고수준 층에서 활성화 안에 상관관계를 비슷하게 유지한다. 특성의 상관관계는 텍스처를 나타낸다. 따라서 생성된 이미지와 스타일 참조 이미지는 여러 크기의 텍스처를 공유할 것이다. <br>
- 게티스 등은 활성화 출력의 <b>Gram Matrix(그람 행렬)</b>을 스타일 손실로 사용했다. <br><br>


#### 12.3.3 케라스로 뉴럴 스타일 트랜스퍼 구현하기
뉴럴 스타일 트랜스퍼는 사전 훈련된 컨브넷 중 어떤 것을 사용해서도 구현할 수 있다. VGG16 네트워크의 변종으로 합성곱 층이 3개 더 추가되었다. 일반적인 과정은 다음과 같다. <br>

1. 스타일 참조 이미지, 베이스 이미지 (base image), 생성된 이미지를 위해 VGG19의 층 활성화를 동시에 계산하는 네트워크를 설정한다. <br>
2. 세 이미지에서 계산한 층 활성화를 사용하여 앞서 설명한 손실 함수를 정의한다. 이 손실을 최소화하여 스타일 트랜스퍼를 구현할 것이다. <br>
3. 손실 함수를 최소화할 경사 하강법 과정을 설정한다. <br>


#### 코드 12-16 스타일 이미지와 콘텐츠 이미지 준비하기

In [ ]:
from tensorflow import keras

base_image_path = keras.utils.get_file( # 변환할 이미지 경로
    "sf.jpg", origin = "https://img-datasets.s3.amazonaws.com/sf.jpg")
style_reference_image_path = keras.utils.get_file(  # 스타일 이미지 경로
    "starry_night.jpg", origin = "https://img-datasets.s3.amazonaws.com/starry_night.jpg")

original_width, original_height = keras.utils.load_img(base_image_path).size
img_height = 400  # 생성 이미지의 차원
img_width = round(original_width*img_height/original_height)  

#### 코드 12-17 유틸리티 함수

In [ ]:
import numpy as np

def preprocess_image(image_path):  # 이미지를 로드하고, 크기를 바꾸어 적절한 배열로 변환하는 유틸리티 함수
    img = keras.utils.load_img(
        image_path, target_size=(img_height, img_width))
    img = keras.utils.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = keras.applications.vgg19.preprocess_input(img)
    return img


def deprocess_image(img):  # 넘파이 배열을 이미지로 변환하는 유틸리티 함수
    img = img.reshape((img_height, img_width, 3))
    # ImageNet의 평균 픽셀값을 더한다. 이는 vgg19.preprocess_input 함수에서 수행한 변환을 복원한다.
    img[:, :, 0] += 103.939
    img[:, :, 1] += 116.779
    img[:, :, 2] += 123.68
    # 이미지를 'BGR'에서 'RGB'로 변환한다. 이것도 vgg19.preprocess_input 함수에서 수행한 변환을 복원하기 위해서다.
    img = img[:, :, ::-1] 
    img = np.clip(img, 0, 255).astype("uint8")
    return img

VGG19 네트워크를 준비, 사전 훈련된 컨브넷을 사용하여 중간층의 활성화를 반환하는 특성 추출 모델을 만들겠다. <br>
이번에는 모델에 있는 모든 층을 사용한다.

#### 코드 12-18 사전 훈련된 VGG19모델을 사용해서 특성 추출기 만들기

In [ ]:
model = keras.applications.vgg19.VGG19(weights="imagenet", include_top=False)  # ImageNet에서 사전 훈련된 가중치로 VGG19 모델을 만든다.
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
feature_extractor = keras.Model(inputs=model.inputs, outputs=outputs_dict)
# 이 모델은 모든 타깃 층의 활성화 값을 (하나의 딕셔너리로) 반환한다.

#### 코드 12-19 콘텐츠 사실

In [ ]:
def content_loss(base_img, combination_img):
    return tf.reduce_sum(tf.square(combination_img - base_img))

#### 코드 12-20 스타일 손실

In [ ]:
def gram_matrix(x):
    x = tf.transpose(x, (2, 0, 1))
    features = tf.reshape(x, (tf.shape(x)[0], -1))
    gram = tf.matmul(features, tf.transpose(features))
    return gram

def style_loss(style_img, combination_img):
    S = gram_matrix(style_img)
    C = gram_matrix(combination_img)
    channels = 3
    size = img_height * img_width
    return tf.reduce_sum(tf.square(S-C))/(4.0*(channels**2)*(size**2))

두 손실에 하나를 더 추가한다. 생성된 이미지의 픽셀을 사용하여 계싼하는 총 변위 손실(total variation loss)이다. 이는 생성된 이미지가 공간적인 연속성을 가지도록 도와주며 픽셀의 격자 무늬가 과도하게 나타나는 것을 막아준다.

#### 코드 12-21 총 변위 손실

In [ ]:
def total_variation_loss(x):
    a = tf.square(
            x[:, :img_height-1, :img_width -1, :] - x[:, 1:, :img_width-1, :]
    )
    b = tf.square(
            x[:, :img_height-1, :img_width -1, :] - x[:, :img_width-1, 1:, :]
    )
    
    return tf.reduce_sum(tf.pow(a+b, 1.25))

#### 코드 12-22  최소화할 최종 손실 정의하기

In [ ]:
style_layer_names = [  # 스타일 손실에 사용할 층
    "block1_conv1",
    "block2_conv1",
    "block3_conv1",
    "block4_conv1",
    "block5_conv1",
]
content_layer_name = "block5_conv2"  # 콘텐츠 손실에 사용할 층
total_variation_weight = 1e-6        # 총 변이 손실의 기여 가중치
style_weight = 1e-6                  # 스타일 손실의 기여 가중치
content_weight = 2.5e-8              # 콘텐츠 손실의 기여 가중치

def compute_loss(combination_image, base_image, style_reference_image):
    input_tensor = tf.concat([base_image, style_reference_image, combination_image], axis=0)
    features = feature_extractor(input_tensor)
    loss = tf.zeros(shape=())  # 손실을 0으로 초기화
    
    # 콘텐츠 손실을 더한다.
    layer_features = features[content_layer_name]
    base_image_features = layer_features[0, :, :, :]
    combination_features = layer_features[2, :, :, :]
    loss = loss + content_weight * content_loss(
        base_image_features, combination_features
    )
    
    # 스타일 손실을 더한다.
    for layer_name in style_layer_names:
        layer_features = features[layer_name]
        style_reference_features = layer_features[1, :, :, :]
        combination_features = layer_features[2, :, :, :]
        style_loss_value = style_loss(style_reference_features, combination_features)
        loss += (style_weight/len(style_layer_names))*style_loss_value
        
    loss += total_variation_weight * total_variation_loss(combination_image)  # 총 변위 손실을 더한다
    return loss

#### 코드 12-23 경사 하강법 단계 설정하기

In [ ]:
import tensorflow as tf

@tf.function  # tf.function으ㅗ 컴파일하여 훈련 스텝의 속도를 높임
def compute_loss_and_grads(
    combination_image, base_image, style_reference_image):
    with tf.GradientTape() as tape:
        loss = compute_loss(
        combination_image, base_image, style_reference_image)
    grads = tape.gradient(loss, combination_image)
    return loss, grads